In [1]:
from glob import glob
from os import path
import seaborn as sns
import json
import pandas as pd
import os
import numpy as np
import re
from os.path import basename, splitext
import time

# from solver import Instance

from argparse import Namespace
import sys
sys.path.append('../solver')
# from solver_output import practice_print
from solver_rostering_w_run_limits import run_roster_solver_objval

/home/clarice/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
list_city_dt = []

for file_in in os.listdir(r'../instances'):
    base_file = splitext(basename(file_in))[0]
    base_file_split = base_file.split('_')
    file = base_file_split[0]+'_'+base_file_split[1]
    list_city_dt.append(file)

list_city_dt = list(set(list_city_dt))
list_city_dt.sort()
print(list_city_dt)
print(len(list_city_dt))


['berlin_db=0.50', 'berlin_db=1.00', 'berlin_db=2.00', 'berlin_db=4.00', 'frankfurt_db=0.50', 'frankfurt_db=1.00', 'frankfurt_db=2.00', 'frankfurt_db=4.00', 'lyon_db=0.50', 'lyon_db=1.00', 'lyon_db=2.00', 'lyon_db=4.00', 'paris_db=0.50', 'paris_db=1.00', 'paris_db=2.00', 'paris_db=4.00']
16


In [3]:
#Subset to city

list_city_dt_subset = []

for city_db in list_city_dt:
    if (city_db.find('lyon') > -1):
        list_city_dt_subset.append(city_db)

print(list_city_dt_subset)

['lyon_db=0.50', 'lyon_db=1.00', 'lyon_db=2.00', 'lyon_db=4.00']


In [4]:

def dict_append_workforce(dict_, base_file, city, demand_baseline, OC, RM, GM, model, max_n_shifts, workforce_size, objvalprev, objval, objvalnext, run_time):
    dict_['instance_file_base'].append(base_file)
    dict_['city'].append(city)
    dict_['demand_baseline'].append(demand_baseline)
    dict_['model'].append(model)
    dict_['max_n_shifts'].append(max_n_shifts)
    dict_['outsourcing_cost_multiplier'].append(OC)
    dict_['regional_multiplier'].append(RM)
    dict_['global_multiplier'].append(GM)
    dict_['workforce_size_region0'].append(workforce_size)
    dict_['objective_value_prev'].append(objvalprev)
    dict_['objective_value'].append(objval)
    dict_['objective_value_next'].append(objvalnext)
    dict_['run_time'].append(run_time)
    return dict_

In [16]:
objvalprev = 1.100000000001
objval = 1

objval = round(objval, 2)
objvalprev = round(objvalprev, 2)

if (objval < objvalprev):
    print("yup")



yup


In [34]:
def load_shift(shift_file, OC, RM, GM, model, max_n_shifts = None):
    with open(shift_file, 'r') as file:
        data = json.load(file)
        df_shifts = pd.DataFrame(data)
    df_shifts = df_shifts[(df_shifts['outsourcing_cost_multiplier']==OC)&(df_shifts['regional_multiplier']==RM)&(df_shifts['global_multiplier']==GM)]
    #fixed or flex
    if model in ['fixed','flex']:
        df_shifts = df_shifts[df_shifts['model']==model]
    #partflex
    else:
        df_shifts = df_shifts[(df_shifts['model']==model)&(df_shifts['max_n_shifts']==max_n_shifts)]
    df_shifts.reset_index(drop = True, inplace=True)
    df_shifts = df_shifts[['region','shifts_start']]
    dict_shifts = {}
    for region in df_shifts['region'].unique().tolist():
        dict_shifts[region] = []
        dict_ = df_shifts[df_shifts['region']==region]['shifts_start'].tolist()[0]
        for shift_start in dict_.keys():
            dict_shifts[region].append(dict_[shift_start])
    return dict_shifts

In [54]:
def create_run_dictionary(base_file, OC, RM, GM):

    #create inputs to run a trial workforce size
    weekday_ = f'{base_file}_dt=doublepeak.json'
    weekend_ = f'{base_file}_dt=uniform.json'

    shift_file_weekday = f"../shifts/{weekday_}"
    shift_file_weekend = f"../shifts/{weekend_}"

    #load all shifts
    dict_weekday = {}
    dict_weekend = {}
    dict_track = {}
    dict_opt = {}
    for model in ['fixed','flex','partflex']:
        if model == 'partflex':
            for max_n_shifts in range(2,5):
                dict_ = load_shift(shift_file_weekday, OC, RM, GM, model, max_n_shifts)
                dict_weekday[(model, max_n_shifts)] = dict_
                dict_ = load_shift(shift_file_weekend, OC, RM, GM, model, max_n_shifts)
                dict_weekend[(model, max_n_shifts)] = dict_
                dict_track[(model, max_n_shifts)] = []
                dict_opt[(model, max_n_shifts)] = 0
        else:
            dict_ = load_shift(shift_file_weekday, OC, RM, GM, model, max_n_shifts = None)
            dict_weekday[(model, None)] = dict_
            dict_ = load_shift(shift_file_weekend, OC, RM, GM, model, max_n_shifts = None)
            dict_weekend[(model, None)] = dict_
            dict_track[(model, None)] = []
            dict_opt[(model, None)] = 0

    #create a dictionary of distinct list of (model, max_n_shifts) that are the same

    list_models = [('fixed', None),('flex', None),('partflex',2),('partflex',3),('partflex',4)]
    list_ran = []
    for comb in list_models:
        list_ = list_models.copy()
        list_.remove(comb)
        for other_comb in list_:
            if (dict_weekday[comb]==dict_weekday[other_comb])&(dict_weekend[comb]==dict_weekend[other_comb])&(comb not in list_ran):
                dict_track[comb].append(other_comb)
                list_ran.append(other_comb)

    list_rmv = []
    for comb in dict_track.keys():
        if dict_track[comb] == []:
            list_rmv.append(comb)
    for comb in list_rmv:
        del dict_track[comb]

    return dict_track

In [55]:

base_file = 'lyon_db=0.50'
RM = 1.5
GM = 0.8


dict_track = create_run_dictionary(base_file, OC, RM, GM)

for comb in dict_track.keys():
    print(comb[0])
    print(comb[1])

    #run optimal workforce size and log

    for val in dict_track[comb]:
        print(val[0])
        print(val[1])

        #log for these values as well

fixed
None
partflex
2
flex
None
partflex
3
partflex
4


In [5]:
#Code that determines optimal workforce size

MAX_TRIES = 200

RM = 1.5
GM = 0.8
h_min = 32
h_max = 48
max_n_diff = 3

for base_file in list_city_dt_subset:
# for base_file in ['berlin_db=0.50']:

    if f'{base_file}.json' not in os.listdir(r'../rerun_workforce_size'):

        list_results = []

        #create inputs to run a trial workforce size
        weekday_ = f'{base_file}_dt=doublepeak.json'
        weekend_ = f'{base_file}_dt=uniform.json'

        instance_file_weekday = f"../instances/{weekday_}"
        instance_file_weekend = f"../instances/{weekend_}"

        shift_file_weekday = f"../shifts/{weekday_}"
        shift_file_weekend = f"../shifts/{weekend_}"

        #create dictionary for evaluating optimal workforce size
        workforce_out = f"../rerun_workforce_size/{base_file}.json"
        dict_out = {
            'instance_file_base':[],
            'city':[],
            'demand_baseline':[],
            'model':[],
            'max_n_shifts':[],
            'outsourcing_cost_multiplier':[],
            'regional_multiplier':[],
            'global_multiplier':[],
            'workforce_size_region0':[],
            'objective_value_prev':[],
            'objective_value':[],
            'objective_value_next':[],
            'run_time':[]
        }

        city_pattern = r'(\w+)_db'
        db_pattern = r'db=(\d+\.\d+)'

        city_match = re.search(city_pattern, base_file)
        db_match = re.search(db_pattern, base_file)

        city = city_match.group(1) if city_match else None
        demand_baseline = float(db_match.group(1)) if db_match else None

        #Set min_tries based on demand_baseline
        if demand_baseline == 0.5:
            MIN_TRIES = 5
        elif demand_baseline == 1.0:
            MIN_TRIES = 10
        elif demand_baseline == 2.0:
            MIN_TRIES = 20
        elif demand_baseline == 4.0:
            MIN_TRIES = 50

        for OC in [1.2, 1.5, 1.8, 2.0, 2.5]:
        # for OC in [1.2]:

            dict_run = create_run_dictionary(base_file, OC, RM, GM)
            for comb in dict_run.keys():
                model = comb[0]
                max_n_shifts = comb[1]

                #find the optimal workforce size

                objvalprev = 1e8
                objval = 1e7
                for trial_size in range(MIN_TRIES, MAX_TRIES):
                    workforce_dict = {0:trial_size, 1:1, 2:1, 3:1, 4:1}
                    if (objval < objvalprev)&(np.isnan(objval) == False):
                        objvalprev = objval
                        start_time = time.time()
                        dict_results = run_roster_solver_objval(model, instance_file_weekday, shift_file_weekday, instance_file_weekend, shift_file_weekend, workforce_dict, OC, RM, GM, h_min, h_max, max_n_diff, max_n_shifts)
                        end_time = time.time()
                        run_time = end_time - start_time
                        objval = dict_results['objective_value'][0]
                        objval = round(objval, 2)
                        print(f'DB: {demand_baseline}, OC: {OC}, model: {model}-{max_n_shifts}, trial_size: {trial_size}, run_time: {run_time}')
                        df_ = pd.DataFrame(dict_results)
                        df_['workforce_size_trial'] = trial_size
                        df_['run_time'] = run_time
                        list_results.append(df_)
                        for subcomb in dict_run[comb]:
                            df_['model'] = subcomb[0]
                            df_['max_n_shifts'] = subcomb[1]
                            list_results.append(df_)
                    else:
                        #keep track of optimal
                        dict_out = dict_append_workforce(dict_out, base_file, city, demand_baseline, OC, RM, GM, model, max_n_shifts, trial_size-1, np.nan, objvalprev, objval, run_time)
                        for subcomb in dict_run[comb]:
                            model = subcomb[0]
                            max_n_shifts = subcomb[1]
                            dict_out = dict_append_workforce(dict_out, base_file, city, demand_baseline, OC, RM, GM, model, max_n_shifts, trial_size-1, np.nan, objvalprev, objval, run_time)
                        break
                    if trial_size == int(MAX_TRIES-1):
                        #keep track of optimal
                        dict_out = dict_append_workforce(dict_out, base_file, city, demand_baseline, OC, RM, GM, model, max_n_shifts, trial_size, objvalprev, objval, np.nan, run_time)
                        for subcomb in dict_run[comb]:
                            model = subcomb[0]
                            max_n_shifts = subcomb[1]
                            dict_out = dict_append_workforce(dict_out, base_file, city, demand_baseline, OC, RM, GM, model, max_n_shifts, trial_size-1, np.nan, objvalprev, objval, run_time)

        with open(workforce_out, 'w') as f:
            json.dump(dict_out, f, indent=2)
        df_trials = pd.concat(list_results, ignore_index = True)
        df_trials.to_excel(f'../rerun_workforce_size/{base_file}_all_trials.xlsx', index = False)


Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-06
DB: 2.0, OC: 1.2, model: fixed, trial_size: 40, run_time: 6.8331427574157715
DB: 2.0, OC: 1.2, model: fixed, trial_size: 41, run_time: 6.827777862548828
DB: 2.0, OC: 1.2, model: fixed, trial_size: 42, run_time: 6.7491843700408936
DB: 2.0, OC: 1.2, model: fixed, trial_size: 43, run_time: 6.708173751831055
DB: 2.0, OC: 1.2, model: fixed, trial_size: 44, run_time: 7.005271673202515
DB: 2.0, OC: 1.2, model: fixed, trial_size: 45, run_time: 7.263382196426392
DB: 2.0, OC: 1.2, model: fixed, trial_size: 46, run_time: 7.17313027381897
DB: 2.0, OC: 1.2, model: fixed, trial_size: 47, run_time: 7.573758840560913
DB: 2.0, OC: 1.2, model: fixed, trial_size: 48, run_time: 7.509249687194824
DB: 2.0, OC: 1.2, model: fixed, trial_size: 49, run_time: 7.674404144287109
DB: 2.0, OC: 1.2, model: fixed, trial_size: 50, run_time: 7.868780851364136
DB: 2.0, OC: 1.2, model: fixed, trial_size: 51, run_time: 7.733994483947

- Fix combinations where MIN_TRIES was set too high

berlin_db=1.00	OC = 2.5	flex                            
berlin_db=1.00	OC = 2.5	partflex	3.0
berlin_db=1.00	OC = 2.5	partflex	4.0	
berlin_db=4.00	OC = 2.5	flex                            
berlin_db=4.00	OC = 2.5	partflex	3.0	
berlin_db=4.00	OC = 2.5	partflex	4.0	

In [25]:
#Fix berlin_db=1.00, OC = 2.5, model = flex

base_file = 'berlin_db=1.00'
OC = 2.5
model = 'flex'
MIN_TRIES = 1
list_results = []

MAX_TRIES = 100

RM = 1.5
GM = 0.8
h_min = 32
h_max = 48
max_n_diff = 3

#create inputs to run a trial workforce size
weekday_ = f'{base_file}_dt=doublepeak.json'
weekend_ = f'{base_file}_dt=uniform.json'

instance_file_weekday = f"../instances/{weekday_}"
instance_file_weekend = f"../instances/{weekend_}"

shift_file_weekday = f"../shifts/{weekday_}"
shift_file_weekend = f"../shifts/{weekend_}"

#create dictionary for evaluating optimal workforce size
workforce_out = f"../workforce_size/{base_file}.json"
dict_out = {
    'instance_file_base':[],
    'city':[],
    'demand_baseline':[],
    'model':[],
    'max_n_shifts':[],
    'outsourcing_cost_multiplier':[],
    'regional_multiplier':[],
    'global_multiplier':[],
    'workforce_size_region0':[],
    'objective_value_prev':[],
    'objective_value':[],
    'objective_value_next':[]
}

city_pattern = r'(\w+)_db'
db_pattern = r'db=(\d+\.\d+)'

city_match = re.search(city_pattern, base_file)
db_match = re.search(db_pattern, base_file)

city = city_match.group(1) if city_match else None
demand_baseline = float(db_match.group(1)) if db_match else None

objvalprev = 1e20
objval = 1e19
for trial_size in range(MIN_TRIES, MAX_TRIES):
    workforce_dict = {0:trial_size, 1:1, 2:1, 3:1, 4:1}
    if (objval < objvalprev)&(np.isnan(objval) == False):
        objvalprev = objval
        start_time = time.time()
        dict_results = run_roster_solver_objval(model, instance_file_weekday, shift_file_weekday, instance_file_weekend, shift_file_weekend, workforce_dict, OC, RM, GM, h_min, h_max, max_n_diff)
        end_time = time.time()
        run_time = end_time - start_time
        objval = dict_results['objective_value'][0]
        print(f'DB: {demand_baseline}, OC: {OC}, model: {model}-{max_n_shifts}, trial_size: {trial_size}, run_time: {run_time}')
        df_ = pd.DataFrame(dict_results)
        df_['workforce_size_trial'] = trial_size
        list_results.append(df_)
    else:
        #keep track of optimal
        dict_out = dict_append_workforce(dict_out, base_file, city, demand_baseline, OC, RM, GM, model, max_n_shifts, trial_size-1, np.nan, objvalprev, objval)
        break
    if trial_size == int(MAX_TRIES-1):
        #keep track of optimal
        dict_out = dict_append_workforce(dict_out, base_file, city, demand_baseline, OC, RM, GM, model, max_n_shifts, trial_size, objvalprev, objval, np.nan)

df_replace = pd.concat(list_results, ignore_index = True)

#update optimal workforce size JSON and xlsx

#re-import in the dictionary of optimal workforce sizes
with open(f'../workforce_size/{base_file}.json') as f:
    data = json.load(f)
df_data = pd.DataFrame(data)

#update the corrupt one
df_data.loc[(df_data['city']==city)&(df_data['demand_baseline']==demand_baseline)&(df_data['outsourcing_cost_multiplier']==OC)&(df_data['model']==model), 'workforce_size_region0'] = dict_out['workforce_size_region0'][0]
df_data.loc[(df_data['city']==city)&(df_data['demand_baseline']==demand_baseline)&(df_data['outsourcing_cost_multiplier']==OC)&(df_data['model']==model), 'objective_value_prev'] = dict_out['objective_value_prev'][0]
df_data.loc[(df_data['city']==city)&(df_data['demand_baseline']==demand_baseline)&(df_data['outsourcing_cost_multiplier']==OC)&(df_data['model']==model), 'objective_value'] = dict_out['objective_value'][0]
df_data.loc[(df_data['city']==city)&(df_data['demand_baseline']==demand_baseline)&(df_data['outsourcing_cost_multiplier']==OC)&(df_data['model']==model), 'objective_value_next'] = dict_out['objective_value_next'][0]

#re-export the dictionary of optimal workforce sizes
dict_data = {}
for col in df_data.columns:
    dict_data[col] = df_data[col].tolist()

workforce_out = f'../workforce_size/{base_file}.json'
with open(workforce_out, 'w') as f:
    json.dump(dict_data, f, indent=2)

#re-import in the excel of the trial sizes
base_file_xlsx = f'{base_file}_all_trials.xlsx'
if base_file_xlsx in os.listdir(r'../workforce_size'):
    df_data = pd.read_excel(f'../workforce_size/{base_file_xlsx}')

#update the corrupt records
df_data['keep'] = 1
df_data.loc[(df_data['city']==city)&(df_data['demand_baseline']==demand_baseline)&(df_data['outsourcing_cost_multiplier']==OC)&(df_data['model']==model), 'keep'] = 0
df_data = df_data[df_data['keep']==1]
df_data.drop(columns = ['keep'], inplace = True)
df_data = pd.concat([df_data, df_replace], ignore_index = True)
df_data.sort_values(by = ['city','demand_baseline','outsourcing_cost_multiplier','model','max_n_shifts','workforce_size'], inplace = True)
df_data.reset_index(drop = True, inplace = True)

#re-export the excel of trial sizes
df_data.to_excel(f'../workforce_size/{base_file_xlsx}', index = False)

DB: 1.0, OC: 2.5, model: flex-4, trial_size: 1, run_time: 3.2113425731658936
DB: 1.0, OC: 2.5, model: flex-4, trial_size: 2, run_time: 3.4110374450683594
DB: 1.0, OC: 2.5, model: flex-4, trial_size: 3, run_time: 3.5653791427612305
DB: 1.0, OC: 2.5, model: flex-4, trial_size: 4, run_time: 4.118104934692383
DB: 1.0, OC: 2.5, model: flex-4, trial_size: 5, run_time: 4.21140718460083
DB: 1.0, OC: 2.5, model: flex-4, trial_size: 6, run_time: 4.509648561477661
DB: 1.0, OC: 2.5, model: flex-4, trial_size: 7, run_time: 4.562493085861206
DB: 1.0, OC: 2.5, model: flex-4, trial_size: 8, run_time: 4.75950026512146
DB: 1.0, OC: 2.5, model: flex-4, trial_size: 9, run_time: 4.503172397613525
DB: 1.0, OC: 2.5, model: flex-4, trial_size: 10, run_time: 5.218148231506348
DB: 1.0, OC: 2.5, model: flex-4, trial_size: 11, run_time: 5.376061201095581
DB: 1.0, OC: 2.5, model: flex-4, trial_size: 12, run_time: 5.1968019008636475
DB: 1.0, OC: 2.5, model: flex-4, trial_size: 13, run_time: 5.716201305389404
DB: 1.

In [14]:
#Fix berlin_db=1.00, OC = 2.5, model = partflex-3

base_file = 'berlin_db=1.00'
OC = 2.5
model = 'partflex'
max_n_shifts = 3
MIN_TRIES = 1
list_results = []

MAX_TRIES = 50

RM = 1.5
GM = 0.8
h_min = 32
h_max = 48
max_n_diff = 3

#create inputs to run a trial workforce size
weekday_ = f'{base_file}_dt=doublepeak.json'
weekend_ = f'{base_file}_dt=uniform.json'

instance_file_weekday = f"../instances/{weekday_}"
instance_file_weekend = f"../instances/{weekend_}"

shift_file_weekday = f"../shifts/{weekday_}"
shift_file_weekend = f"../shifts/{weekend_}"

#create dictionary for evaluating optimal workforce size
workforce_out = f"../workforce_size/{base_file}.json"
dict_out = {
    'instance_file_base':[],
    'city':[],
    'demand_baseline':[],
    'model':[],
    'max_n_shifts':[],
    'outsourcing_cost_multiplier':[],
    'regional_multiplier':[],
    'global_multiplier':[],
    'workforce_size_region0':[],
    'objective_value_prev':[],
    'objective_value':[],
    'objective_value_next':[]
}

city_pattern = r'(\w+)_db'
db_pattern = r'db=(\d+\.\d+)'

city_match = re.search(city_pattern, base_file)
db_match = re.search(db_pattern, base_file)

city = city_match.group(1) if city_match else None
demand_baseline = float(db_match.group(1)) if db_match else None

objvalprev = 1e20
objval = 1e19
for trial_size in range(MIN_TRIES, MAX_TRIES):
    workforce_dict = {0:trial_size, 1:1, 2:1, 3:1, 4:1}
    if (objval < objvalprev)&(np.isnan(objval) == False):
        objvalprev = objval
        start_time = time.time()
        dict_results = run_roster_solver_objval(model, instance_file_weekday, shift_file_weekday, instance_file_weekend, shift_file_weekend, workforce_dict, OC, RM, GM, h_min, h_max, max_n_diff, max_n_shifts)
        end_time = time.time()
        run_time = end_time - start_time
        objval = dict_results['objective_value'][0]
        print(f'DB: {demand_baseline}, OC: {OC}, model: {model}-{max_n_shifts}, trial_size: {trial_size}, run_time: {run_time}')
        df_ = pd.DataFrame(dict_results)
        df_['workforce_size_trial'] = trial_size
        list_results.append(df_)
    else:
        #keep track of optimal
        dict_out = dict_append_workforce(dict_out, base_file, city, demand_baseline, OC, RM, GM, model, max_n_shifts, trial_size-1, np.nan, objvalprev, objval)
        break
    if trial_size == int(MAX_TRIES-1):
        #keep track of optimal
        dict_out = dict_append_workforce(dict_out, base_file, city, demand_baseline, OC, RM, GM, model, max_n_shifts, trial_size, objvalprev, objval, np.nan)

df_replace = pd.concat(list_results, ignore_index = True)

#update optimal workforce size JSON and xlsx

#re-import in the dictionary of optimal workforce sizes
with open(f'../workforce_size/{base_file}.json') as f:
    data = json.load(f)
df_data = pd.DataFrame(data)

#update the corrupt one
df_data.loc[(df_data['city']==city)&(df_data['demand_baseline']==demand_baseline)&(df_data['outsourcing_cost_multiplier']==OC)&(df_data['model']==model)&(df_data['max_n_shifts']==max_n_shifts), 'workforce_size_region0'] = dict_out['workforce_size_region0'][0]
df_data.loc[(df_data['city']==city)&(df_data['demand_baseline']==demand_baseline)&(df_data['outsourcing_cost_multiplier']==OC)&(df_data['model']==model)&(df_data['max_n_shifts']==max_n_shifts), 'objective_value_prev'] = dict_out['objective_value_prev'][0]
df_data.loc[(df_data['city']==city)&(df_data['demand_baseline']==demand_baseline)&(df_data['outsourcing_cost_multiplier']==OC)&(df_data['model']==model)&(df_data['max_n_shifts']==max_n_shifts), 'objective_value'] = dict_out['objective_value'][0]
df_data.loc[(df_data['city']==city)&(df_data['demand_baseline']==demand_baseline)&(df_data['outsourcing_cost_multiplier']==OC)&(df_data['model']==model)&(df_data['max_n_shifts']==max_n_shifts), 'objective_value_next'] = dict_out['objective_value_next'][0]

#re-export the dictionary of optimal workforce sizes
dict_data = {}
for col in df_data.columns:
    dict_data[col] = df_data[col].tolist()

workforce_out = f'../workforce_size/{base_file}.json'
with open(workforce_out, 'w') as f:
    json.dump(dict_data, f, indent=2)

#re-import in the excel of the trial sizes
base_file_xlsx = f'{base_file}_all_trials.xlsx'
if base_file_xlsx in os.listdir(r'../workforce_size'):
    df_data = pd.read_excel(f'../workforce_size/{base_file_xlsx}')

#update the corrupt records
df_data['keep'] = 1
df_data.loc[(df_data['city']==city)&(df_data['demand_baseline']==demand_baseline)&(df_data['outsourcing_cost_multiplier']==OC)&(df_data['model']==model)&(df_data['max_n_shifts']==max_n_shifts), 'keep'] = 0
df_data = df_data[df_data['keep']==1]
df_data.drop(columns = ['keep'], inplace = True)
df_data = pd.concat([df_data, df_replace], ignore_index = True)
df_data.sort_values(by = ['city','demand_baseline','outsourcing_cost_multiplier','model','max_n_shifts','workforce_size'], inplace = True)
df_data.reset_index(drop = True, inplace = True)

#re-export the excel of trial sizes
df_data.to_excel(f'../workforce_size/{base_file_xlsx}', index = False)

DB: 1.0, OC: 2.5, model: partflex-3, trial_size: 1, run_time: 3.754678726196289
DB: 1.0, OC: 2.5, model: partflex-3, trial_size: 2, run_time: 3.465759515762329
DB: 1.0, OC: 2.5, model: partflex-3, trial_size: 3, run_time: 3.713054895401001
DB: 1.0, OC: 2.5, model: partflex-3, trial_size: 4, run_time: 3.7161202430725098
DB: 1.0, OC: 2.5, model: partflex-3, trial_size: 5, run_time: 3.8212759494781494
DB: 1.0, OC: 2.5, model: partflex-3, trial_size: 6, run_time: 3.885908842086792
DB: 1.0, OC: 2.5, model: partflex-3, trial_size: 7, run_time: 4.0264503955841064
DB: 1.0, OC: 2.5, model: partflex-3, trial_size: 8, run_time: 4.111868619918823
DB: 1.0, OC: 2.5, model: partflex-3, trial_size: 9, run_time: 4.362640142440796
DB: 1.0, OC: 2.5, model: partflex-3, trial_size: 10, run_time: 4.581350088119507
DB: 1.0, OC: 2.5, model: partflex-3, trial_size: 11, run_time: 4.581140756607056
DB: 1.0, OC: 2.5, model: partflex-3, trial_size: 12, run_time: 4.832568645477295
DB: 1.0, OC: 2.5, model: partflex-

In [24]:
#Fix berlin_db=1.00, OC = 2.5, model = partflex-4

base_file = 'berlin_db=1.00'
OC = 2.5
model = 'partflex'
max_n_shifts = 4
MIN_TRIES = 1
list_results = []

MAX_TRIES = 50

RM = 1.5
GM = 0.8
h_min = 32
h_max = 48
max_n_diff = 3

#create inputs to run a trial workforce size
weekday_ = f'{base_file}_dt=doublepeak.json'
weekend_ = f'{base_file}_dt=uniform.json'

instance_file_weekday = f"../instances/{weekday_}"
instance_file_weekend = f"../instances/{weekend_}"

shift_file_weekday = f"../shifts/{weekday_}"
shift_file_weekend = f"../shifts/{weekend_}"

#create dictionary for evaluating optimal workforce size
workforce_out = f"../workforce_size/{base_file}.json"
dict_out = {
    'instance_file_base':[],
    'city':[],
    'demand_baseline':[],
    'model':[],
    'max_n_shifts':[],
    'outsourcing_cost_multiplier':[],
    'regional_multiplier':[],
    'global_multiplier':[],
    'workforce_size_region0':[],
    'objective_value_prev':[],
    'objective_value':[],
    'objective_value_next':[]
}

city_pattern = r'(\w+)_db'
db_pattern = r'db=(\d+\.\d+)'

city_match = re.search(city_pattern, base_file)
db_match = re.search(db_pattern, base_file)

city = city_match.group(1) if city_match else None
demand_baseline = float(db_match.group(1)) if db_match else None

objvalprev = 1e20
objval = 1e19
for trial_size in range(MIN_TRIES, MAX_TRIES):
    workforce_dict = {0:trial_size, 1:1, 2:1, 3:1, 4:1}
    if (objval < objvalprev)&(np.isnan(objval) == False):
        objvalprev = objval
        start_time = time.time()
        dict_results = run_roster_solver_objval(model, instance_file_weekday, shift_file_weekday, instance_file_weekend, shift_file_weekend, workforce_dict, OC, RM, GM, h_min, h_max, max_n_diff, max_n_shifts)
        end_time = time.time()
        run_time = end_time - start_time
        objval = dict_results['objective_value'][0]
        print(f'DB: {demand_baseline}, OC: {OC}, model: {model}-{max_n_shifts}, trial_size: {trial_size}, run_time: {run_time}')
        df_ = pd.DataFrame(dict_results)
        df_['workforce_size_trial'] = trial_size
        list_results.append(df_)
    else:
        #keep track of optimal
        dict_out = dict_append_workforce(dict_out, base_file, city, demand_baseline, OC, RM, GM, model, max_n_shifts, trial_size-1, np.nan, objvalprev, objval)
        break
    if trial_size == int(MAX_TRIES-1):
        #keep track of optimal
        dict_out = dict_append_workforce(dict_out, base_file, city, demand_baseline, OC, RM, GM, model, max_n_shifts, trial_size, objvalprev, objval, np.nan)

df_replace = pd.concat(list_results, ignore_index = True)

#update optimal workforce size JSON and xlsx

#re-import in the dictionary of optimal workforce sizes
with open(f'../workforce_size/{base_file}.json') as f:
    data = json.load(f)
df_data = pd.DataFrame(data)

#update the corrupt one
df_data.loc[(df_data['city']==city)&(df_data['demand_baseline']==demand_baseline)&(df_data['outsourcing_cost_multiplier']==OC)&(df_data['model']==model)&(df_data['max_n_shifts']==max_n_shifts), 'workforce_size_region0'] = dict_out['workforce_size_region0'][0]
df_data.loc[(df_data['city']==city)&(df_data['demand_baseline']==demand_baseline)&(df_data['outsourcing_cost_multiplier']==OC)&(df_data['model']==model)&(df_data['max_n_shifts']==max_n_shifts), 'objective_value_prev'] = dict_out['objective_value_prev'][0]
df_data.loc[(df_data['city']==city)&(df_data['demand_baseline']==demand_baseline)&(df_data['outsourcing_cost_multiplier']==OC)&(df_data['model']==model)&(df_data['max_n_shifts']==max_n_shifts), 'objective_value'] = dict_out['objective_value'][0]
df_data.loc[(df_data['city']==city)&(df_data['demand_baseline']==demand_baseline)&(df_data['outsourcing_cost_multiplier']==OC)&(df_data['model']==model)&(df_data['max_n_shifts']==max_n_shifts), 'objective_value_next'] = dict_out['objective_value_next'][0]

#re-export the dictionary of optimal workforce sizes
dict_data = {}
for col in df_data.columns:
    dict_data[col] = df_data[col].tolist()

workforce_out = f'../workforce_size/{base_file}.json'
with open(workforce_out, 'w') as f:
    json.dump(dict_data, f, indent=2)

#re-import in the excel of the trial sizes
base_file_xlsx = f'{base_file}_all_trials.xlsx'
if base_file_xlsx in os.listdir(r'../workforce_size'):
    df_data = pd.read_excel(f'../workforce_size/{base_file_xlsx}')

#update the corrupt records
df_data['keep'] = 1
df_data.loc[(df_data['city']==city)&(df_data['demand_baseline']==demand_baseline)&(df_data['outsourcing_cost_multiplier']==OC)&(df_data['model']==model)&(df_data['max_n_shifts']==max_n_shifts), 'keep'] = 0
df_data = df_data[df_data['keep']==1]
df_data.drop(columns = ['keep'], inplace = True)
df_data = pd.concat([df_data, df_replace], ignore_index = True)
df_data.sort_values(by = ['city','demand_baseline','outsourcing_cost_multiplier','model','max_n_shifts','workforce_size'], inplace = True)
df_data.reset_index(drop = True, inplace = True)

#re-export the excel of trial sizes
df_data.to_excel(f'../workforce_size/{base_file_xlsx}', index = False)

DB: 1.0, OC: 2.5, model: partflex-4, trial_size: 1, run_time: 3.757735252380371
DB: 1.0, OC: 2.5, model: partflex-4, trial_size: 2, run_time: 3.6414365768432617
DB: 1.0, OC: 2.5, model: partflex-4, trial_size: 3, run_time: 3.875638723373413
DB: 1.0, OC: 2.5, model: partflex-4, trial_size: 4, run_time: 4.0295090675354
DB: 1.0, OC: 2.5, model: partflex-4, trial_size: 5, run_time: 4.265816926956177
DB: 1.0, OC: 2.5, model: partflex-4, trial_size: 6, run_time: 4.437159061431885
DB: 1.0, OC: 2.5, model: partflex-4, trial_size: 7, run_time: 4.867093801498413
DB: 1.0, OC: 2.5, model: partflex-4, trial_size: 8, run_time: 4.647626876831055
DB: 1.0, OC: 2.5, model: partflex-4, trial_size: 9, run_time: 4.895684003829956
DB: 1.0, OC: 2.5, model: partflex-4, trial_size: 10, run_time: 5.03456711769104
DB: 1.0, OC: 2.5, model: partflex-4, trial_size: 11, run_time: 4.898409843444824
DB: 1.0, OC: 2.5, model: partflex-4, trial_size: 12, run_time: 4.675569772720337
DB: 1.0, OC: 2.5, model: partflex-4, tr

In [32]:
base_file = 'berlin_db=4.00'
MIN_TRIES = 1
MAX_TRIES = 200

RM = 1.5
GM = 0.8
h_min = 32
h_max = 48
max_n_diff = 3

for OC in [1.2, 1.5, 1.8, 2.0, 2.5]:
    for model in ['flex','partflex']:
        if model == 'partflex':
            for max_n_shifts in range(3,5):
                list_results = []

                #create inputs to run a trial workforce size
                weekday_ = f'{base_file}_dt=doublepeak.json'
                weekend_ = f'{base_file}_dt=uniform.json'

                instance_file_weekday = f"../instances/{weekday_}"
                instance_file_weekend = f"../instances/{weekend_}"

                shift_file_weekday = f"../shifts/{weekday_}"
                shift_file_weekend = f"../shifts/{weekend_}"

                #create dictionary for evaluating optimal workforce size
                workforce_out = f"../workforce_size/{base_file}.json"
                dict_out = {
                    'instance_file_base':[],
                    'city':[],
                    'demand_baseline':[],
                    'model':[],
                    'max_n_shifts':[],
                    'outsourcing_cost_multiplier':[],
                    'regional_multiplier':[],
                    'global_multiplier':[],
                    'workforce_size_region0':[],
                    'objective_value_prev':[],
                    'objective_value':[],
                    'objective_value_next':[]
                }

                city_pattern = r'(\w+)_db'
                db_pattern = r'db=(\d+\.\d+)'

                city_match = re.search(city_pattern, base_file)
                db_match = re.search(db_pattern, base_file)

                city = city_match.group(1) if city_match else None
                demand_baseline = float(db_match.group(1)) if db_match else None

                objvalprev = 1e20
                objval = 1e19
                for trial_size in range(MIN_TRIES, MAX_TRIES):
                    workforce_dict = {0:trial_size, 1:1, 2:1, 3:1, 4:1}
                    if (objval < objvalprev)&(np.isnan(objval) == False):
                        objvalprev = objval
                        start_time = time.time()
                        dict_results = run_roster_solver_objval(model, instance_file_weekday, shift_file_weekday, instance_file_weekend, shift_file_weekend, workforce_dict, OC, RM, GM, h_min, h_max, max_n_diff, max_n_shifts)
                        end_time = time.time()
                        run_time = end_time - start_time
                        objval = dict_results['objective_value'][0]
                        print(f'DB: {demand_baseline}, OC: {OC}, model: {model}-{max_n_shifts}, trial_size: {trial_size}, run_time: {run_time}')
                        df_ = pd.DataFrame(dict_results)
                        df_['workforce_size_trial'] = trial_size
                        list_results.append(df_)
                    else:
                        #keep track of optimal
                        dict_out = dict_append_workforce(dict_out, base_file, city, demand_baseline, OC, RM, GM, model, max_n_shifts, trial_size-1, np.nan, objvalprev, objval)
                        break
                    if trial_size == int(MAX_TRIES-1):
                        #keep track of optimal
                        dict_out = dict_append_workforce(dict_out, base_file, city, demand_baseline, OC, RM, GM, model, max_n_shifts, trial_size, objvalprev, objval, np.nan)

                df_replace = pd.concat(list_results, ignore_index = True)

                #update optimal workforce size JSON and xlsx

                #re-import in the dictionary of optimal workforce sizes
                with open(f'../workforce_size/{base_file}.json') as f:
                    data = json.load(f)
                df_data = pd.DataFrame(data)

                #update the corrupt one
                df_data.loc[(df_data['city']==city)&(df_data['demand_baseline']==demand_baseline)&(df_data['outsourcing_cost_multiplier']==OC)&(df_data['model']==model)&(df_data['max_n_shifts']==max_n_shifts), 'workforce_size_region0'] = dict_out['workforce_size_region0'][0]
                df_data.loc[(df_data['city']==city)&(df_data['demand_baseline']==demand_baseline)&(df_data['outsourcing_cost_multiplier']==OC)&(df_data['model']==model)&(df_data['max_n_shifts']==max_n_shifts), 'objective_value_prev'] = dict_out['objective_value_prev'][0]
                df_data.loc[(df_data['city']==city)&(df_data['demand_baseline']==demand_baseline)&(df_data['outsourcing_cost_multiplier']==OC)&(df_data['model']==model)&(df_data['max_n_shifts']==max_n_shifts), 'objective_value'] = dict_out['objective_value'][0]
                df_data.loc[(df_data['city']==city)&(df_data['demand_baseline']==demand_baseline)&(df_data['outsourcing_cost_multiplier']==OC)&(df_data['model']==model)&(df_data['max_n_shifts']==max_n_shifts), 'objective_value_next'] = dict_out['objective_value_next'][0]

                #re-export the dictionary of optimal workforce sizes
                dict_data = {}
                for col in df_data.columns:
                    dict_data[col] = df_data[col].tolist()

                workforce_out = f'../workforce_size/{base_file}.json'
                with open(workforce_out, 'w') as f:
                    json.dump(dict_data, f, indent=2)

                #re-import in the excel of the trial sizes
                base_file_xlsx = f'{base_file}_all_trials.xlsx'
                if base_file_xlsx in os.listdir(r'../workforce_size'):
                    df_data = pd.read_excel(f'../workforce_size/{base_file_xlsx}')

                #update the corrupt records
                df_data['keep'] = 1
                df_data.loc[(df_data['city']==city)&(df_data['demand_baseline']==demand_baseline)&(df_data['outsourcing_cost_multiplier']==OC)&(df_data['model']==model)&(df_data['max_n_shifts']==max_n_shifts), 'keep'] = 0
                df_data = df_data[df_data['keep']==1]
                df_data.drop(columns = ['keep'], inplace = True)
                df_data = pd.concat([df_data, df_replace], ignore_index = True)
                df_data.sort_values(by = ['city','demand_baseline','outsourcing_cost_multiplier','model','max_n_shifts','workforce_size'], inplace = True)
                df_data.reset_index(drop = True, inplace = True)

                #re-export the excel of trial sizes
                df_data.to_excel(f'../workforce_size/{base_file_xlsx}', index = False)
        else:
            list_results = []

            #create inputs to run a trial workforce size
            weekday_ = f'{base_file}_dt=doublepeak.json'
            weekend_ = f'{base_file}_dt=uniform.json'

            instance_file_weekday = f"../instances/{weekday_}"
            instance_file_weekend = f"../instances/{weekend_}"

            shift_file_weekday = f"../shifts/{weekday_}"
            shift_file_weekend = f"../shifts/{weekend_}"

            #create dictionary for evaluating optimal workforce size
            workforce_out = f"../workforce_size/{base_file}.json"
            dict_out = {
                'instance_file_base':[],
                'city':[],
                'demand_baseline':[],
                'model':[],
                'max_n_shifts':[],
                'outsourcing_cost_multiplier':[],
                'regional_multiplier':[],
                'global_multiplier':[],
                'workforce_size_region0':[],
                'objective_value_prev':[],
                'objective_value':[],
                'objective_value_next':[]
            }

            city_pattern = r'(\w+)_db'
            db_pattern = r'db=(\d+\.\d+)'

            city_match = re.search(city_pattern, base_file)
            db_match = re.search(db_pattern, base_file)

            city = city_match.group(1) if city_match else None
            demand_baseline = float(db_match.group(1)) if db_match else None

            objvalprev = 1e20
            objval = 1e19
            for trial_size in range(MIN_TRIES, MAX_TRIES):
                workforce_dict = {0:trial_size, 1:1, 2:1, 3:1, 4:1}
                if (objval < objvalprev)&(np.isnan(objval) == False):
                    objvalprev = objval
                    start_time = time.time()
                    dict_results = run_roster_solver_objval(model, instance_file_weekday, shift_file_weekday, instance_file_weekend, shift_file_weekend, workforce_dict, OC, RM, GM, h_min, h_max, max_n_diff)
                    end_time = time.time()
                    run_time = end_time - start_time
                    objval = dict_results['objective_value'][0]
                    print(f'DB: {demand_baseline}, OC: {OC}, model: {model}, trial_size: {trial_size}, run_time: {run_time}')
                    df_ = pd.DataFrame(dict_results)
                    df_['workforce_size_trial'] = trial_size
                    list_results.append(df_)
                else:
                    #keep track of optimal
                    dict_out = dict_append_workforce(dict_out, base_file, city, demand_baseline, OC, RM, GM, model, max_n_shifts, trial_size-1, np.nan, objvalprev, objval)
                    break
                if trial_size == int(MAX_TRIES-1):
                    #keep track of optimal
                    dict_out = dict_append_workforce(dict_out, base_file, city, demand_baseline, OC, RM, GM, model, max_n_shifts, trial_size, objvalprev, objval, np.nan)

            df_replace = pd.concat(list_results, ignore_index = True)

            #update optimal workforce size JSON and xlsx

            #re-import in the dictionary of optimal workforce sizes
            with open(f'../workforce_size/{base_file}.json') as f:
                data = json.load(f)
            df_data = pd.DataFrame(data)

            #update the corrupt one
            df_data.loc[(df_data['city']==city)&(df_data['demand_baseline']==demand_baseline)&(df_data['outsourcing_cost_multiplier']==OC)&(df_data['model']==model), 'workforce_size_region0'] = dict_out['workforce_size_region0'][0]
            df_data.loc[(df_data['city']==city)&(df_data['demand_baseline']==demand_baseline)&(df_data['outsourcing_cost_multiplier']==OC)&(df_data['model']==model), 'objective_value_prev'] = dict_out['objective_value_prev'][0]
            df_data.loc[(df_data['city']==city)&(df_data['demand_baseline']==demand_baseline)&(df_data['outsourcing_cost_multiplier']==OC)&(df_data['model']==model), 'objective_value'] = dict_out['objective_value'][0]
            df_data.loc[(df_data['city']==city)&(df_data['demand_baseline']==demand_baseline)&(df_data['outsourcing_cost_multiplier']==OC)&(df_data['model']==model), 'objective_value_next'] = dict_out['objective_value_next'][0]

            #re-export the dictionary of optimal workforce sizes
            dict_data = {}
            for col in df_data.columns:
                dict_data[col] = df_data[col].tolist()

            workforce_out = f'../workforce_size/{base_file}.json'
            with open(workforce_out, 'w') as f:
                json.dump(dict_data, f, indent=2)

            #re-import in the excel of the trial sizes
            base_file_xlsx = f'{base_file}_all_trials.xlsx'
            if base_file_xlsx in os.listdir(r'../workforce_size'):
                df_data = pd.read_excel(f'../workforce_size/{base_file_xlsx}')

            #update the corrupt records
            df_data['keep'] = 1
            df_data.loc[(df_data['city']==city)&(df_data['demand_baseline']==demand_baseline)&(df_data['outsourcing_cost_multiplier']==OC)&(df_data['model']==model), 'keep'] = 0
            df_data = df_data[df_data['keep']==1]
            df_data.drop(columns = ['keep'], inplace = True)
            df_data = pd.concat([df_data, df_replace], ignore_index = True)
            df_data.sort_values(by = ['city','demand_baseline','outsourcing_cost_multiplier','model','max_n_shifts','workforce_size'], inplace = True)
            df_data.reset_index(drop = True, inplace = True)

            #re-export the excel of trial sizes
            df_data.to_excel(f'../workforce_size/{base_file_xlsx}', index = False)


DB: 4.0, OC: 1.2, model: flex, trial_size: 1, run_time: 3.2592878341674805
DB: 4.0, OC: 1.2, model: flex, trial_size: 2, run_time: 3.6037027835845947
DB: 4.0, OC: 1.2, model: flex, trial_size: 3, run_time: 3.578697681427002
DB: 4.0, OC: 1.2, model: flex, trial_size: 4, run_time: 3.7137510776519775
DB: 4.0, OC: 1.2, model: flex, trial_size: 5, run_time: 3.9018075466156006
DB: 4.0, OC: 1.2, model: flex, trial_size: 6, run_time: 4.3038763999938965
DB: 4.0, OC: 1.2, model: flex, trial_size: 7, run_time: 4.290249347686768
DB: 4.0, OC: 1.2, model: flex, trial_size: 8, run_time: 4.474467992782593
DB: 4.0, OC: 1.2, model: flex, trial_size: 9, run_time: 4.573718309402466
DB: 4.0, OC: 1.2, model: flex, trial_size: 10, run_time: 4.482902526855469
DB: 4.0, OC: 1.2, model: flex, trial_size: 11, run_time: 4.437744140625
DB: 4.0, OC: 1.2, model: flex, trial_size: 12, run_time: 4.395033597946167
DB: 4.0, OC: 1.2, model: flex, trial_size: 13, run_time: 4.498821973800659
DB: 4.0, OC: 1.2, model: flex, t